In [3]:
# 情感词抽取
import jieba.posseg as pseg
import jieba
import re
import pandas as pd
from tqdm import tqdm
# 结巴初始化
jieba.initialize()
aspect_words=pd.read_table('CAT法抽取到的方面词.tsv',encoding='utf-8',header=None)[0].tolist()

# 抽取<方面,情感>词组,并写入文件
stopwords=[line.strip() for line in open('D:\python_common\基本语料库\停用词表完全版.txt',encoding='utf-8').readlines()]
def find_pattern(sentence_cut_,word,index,checked):
    sentence_cut=sentence_cut_.copy()
    # 形容词+属性词
    if index > 0 and pseg.lcut(sentence_cut[index-1])[0].flag in ['a','v'] and index-1 not in checked :
        checked.append(index-1)
        return  [sentence_cut[index-1],word],checked
    # 属性词+形容词
    elif index+1 < len(sentence_cut) and pseg.lcut(sentence_cut[index+1])[0].flag in ['a','x','v'] and index+1 not in checked :
        checked.append(index+1)
        return  [word,sentence_cut[index+1]],checked
    # 属性词+形容词+形容词
    elif index+2 < len(sentence_cut) and pseg.lcut(sentence_cut[index+1])[0].flag in ['a','x','v'] and pseg.lcut(sentence_cut[index+2])[0].flag in ['a','x','v'] and index+2 not in checked:
        checked.append(index+2)
        return  [word,sentence_cut[index+1],sentence_cut[index+2]],checked
    # 形容词+形容词+属性词
    elif index+2 < len(sentence_cut) and pseg.lcut(sentence_cut[index+1])[0].flag in ['a','x','v'] and pseg.lcut(sentence_cut[index+2])[0].flag in ['a','x','v'] and index+2 not in checked:
        checked.append(index+2)
        return  [word,sentence_cut[index+1],sentence_cut[index+2]],checked
    # 属性词+副词+形容词
    elif index+2 < len(sentence_cut) and pseg.lcut(sentence_cut[index+1])[0].flag in ['d','x','zg'] and pseg.lcut(sentence_cut[index+2])[0].flag in ['a','v','x'] and index+2 not in checked :
        checked.append(index+2)
        return  [word,sentence_cut[index+1],sentence_cut[index+2]],checked
    # 副词+形容词+属性词
    elif 2 < index < len(sentence_cut)-1 and pseg.lcut(sentence_cut[index - 2])[0].flag in ['d', 'x'] and pseg.lcut(sentence_cut[index - 1])[0].flag in ['a', 'v', 'x'] and index-2 not in checked:
        checked.append(index-2)
        return  [sentence_cut[index-2],sentence_cut[index-1],word],checked

    return sentence_cut,checked

def check(sentence):
    sentence_cut_=pseg.lcut(sentence.strip())
    sentence_cut=[each.word for each in sentence_cut_ if each.word not in stopwords]
    S=[];checked=[]
    for each_word_ in range(len(sentence_cut)):
        if sentence_cut[each_word_] in aspect_words:
            result=find_pattern(sentence_cut,sentence_cut[each_word_],each_word_,checked)
            if result is not None:
                ss,checked=result
            if ss != sentence_cut:
                S.append(ss)
    if len(S)>=1 and type(S[0])==list:
        return S

    sentence2 = sentence
    sentence2 = re.sub(r'[^\w\s]','',sentence2)
    sentence2 = re.sub(r'\d+', '', sentence2)
    sentence2 = re.sub(r'  ', '', sentence2)
    sentence2 = re.sub(r'[a-zA-Z]+', '', sentence2)
    sentence2 = re.sub(r'.件', '', sentence2)
    sentence2 = sentence2.strip()

    sentence_cut2=jieba.lcut(sentence2)
    sentence_cut2=[each for each in sentence_cut2 if each not in stopwords  and each !='\n']

    if len(sentence_cut2) >0:
        S.append(sentence_cut2[0:5])
    if len(S) >0:
        return S
    else:
        S=[]
        [S.append(each) for each in jieba.lcut(sentence.strip()) if each not in S]
        return S

for pinpai in ['ALBD','AND','AT','BNL','GRN','LN','QPL','TB']:
    print(pinpai)
    data=open('爬取结果/爬取结果_'+pinpai+'.txt',encoding='utf-8').readlines()
    with open('D:\python_common\代码_感知质量大论文\公开数据处理\感知质量数据集\关键词抽取结果\关键词抽取结果_'+pinpai+'.tsv','w',encoding='utf-8') as f:
        for each in tqdm(data):
            result=check(each)
            [f.write(''.join(each_r)+'\t'+each.strip()+'\n') for each_r in result]

ALBD


  0%|          | 0/16785 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\93461\AppData\Local\Temp\jieba.cache
Loading model cost 0.717 seconds.
Prefix dict has been built successfully.
100%|██████████| 16785/16785 [00:36<00:00, 461.98it/s]


AND


100%|██████████| 16582/16582 [00:27<00:00, 592.74it/s]


AT


...

100%|██████████| 15981/15981 [00:30<00:00, 526.47it/s]


QPL


100%|██████████| 15727/15727 [00:38<00:00, 412.92it/s]


TB


100%|██████████| 14474/14474 [00:26<00:00, 545.93it/s]
